In [2]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import re

# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [26]:
# Per Samantha Chu:
# When dealing with text data, there are common pre-processing steps. We won't necessarily use all of them every time we deal with text data.
#- Remove special characters (e.g. Regular expression to just keep the words)
#- Tokenizing 
#- Lemmatizing/Stemming
#- Stop word removal

In [3]:
# remove special chars
#df['title'] = df['title'].replace('[^\w ]','',regex=True).astype(str) 
# awesome regex shortcut thanks to https://stackoverflow.com/questions/1219915/regex-to-remove-apostrophe

In [4]:
astro_comm_df = pd.read_csv('./datasets/eda_astro_comment.csv')
quantum_comm_df = pd.read_csv('./datasets/eda_quantum_comment.csv')

Start with additional light cleaning to each DataFrame and then combine them into one (before running train_test_split etc)

**In hindsight this could have been done in the earlier EDA phase** but *c'est la vie*

In [5]:
astro_comm_df.head(2)

,Unnamed: 0,body,created_utc,subreddit,author,Category,Post_comment,word_count
0,0,Your mum says your pole is more like 2 inches ...,1617222727,astrophysics,moon-worshiper,Astrophysics,Comment,67
1,1,"&gt; 760 mph\n\nJust to clarify, thats the spe...",1617220516,astrophysics,Lewri,Astrophysics,Comment,19


In [6]:
astro_comm_df.drop(columns=['Unnamed: 0','created_utc','subreddit'],inplace=True)
astro_comm_df

,body,author,Category,Post_comment,word_count
0,Your mum says your pole is more like 2 inches ...,moon-worshiper,Astrophysics,Comment,67
1,"&gt; 760 mph\n\nJust to clarify, thats the spe...",Lewri,Astrophysics,Comment,19
2,Threw this git repo together [https://github.c...,physmathastro,Astrophysics,Comment,40
3,oh okay thanks for clearing that.,AryanPandey,Astrophysics,Comment,6
4,"again, the movement needs to propagate through...",Lewri,Astrophysics,Comment,24
...,...,...,...,...,...
9476,You can make similar conclusions about redshif...,Tremongulous_Derf,Astrophysics,Comment,53
9477,Trust me -- you can usually busk it. Depends o...,curiousscribbler,Astrophysics,Comment,43
9478,Light has a frequency that can be measured. Wh...,MrMakeItAllUp,Astrophysics,Comment,208
9479,The interplay between story details and the ph...,ketarax,Astrophysics,Comment,48


In [7]:
quantum_comm_df.drop(columns=['Unnamed: 0','created_utc','subreddit'],inplace=True)
quantum_comm_df

,body,author,Category,Post_comment,word_count
0,Related question: there are cloud-based quantu...,1E4rth,Quantum,Comment,89
1,"3-SAT is NP-complete, and we don't expect to b...",crispyleprechaun,Quantum,Comment,98
2,It is currently conjectured that quantum compu...,cirosantilli,Quantum,Comment,34
3,"serious question, since i know nothing about t...",maexx80,Quantum,Comment,57
4,"This was really an excellent podcast, thanks!",doxorubicin2001d,Quantum,Comment,7
...,...,...,...,...,...
9751,[https://www.lanl.gov/projects/national-securi...,WorriedPurpose,Quantum,Comment,1
9752,"Yeah, just thought that post was particularly ...",SaltKick2,Quantum,Comment,33
9753,‘Internships’ are rarely advertised. Find prof...,youngeverest,Quantum,Comment,69
9754,I bet the quantum internet people at TUDelf an...,rrtucci,Quantum,Comment,23


In [17]:
all_comm_df = pd.concat([astro_comm_df,quantum_comm_df])
all_comm_df

,body,author,Category,Post_comment,word_count
0,Your mum says your pole is more like 2 inches ...,moon-worshiper,Astrophysics,Comment,67
1,"&gt; 760 mph\n\nJust to clarify, thats the spe...",Lewri,Astrophysics,Comment,19
2,Threw this git repo together [https://github.c...,physmathastro,Astrophysics,Comment,40
3,oh okay thanks for clearing that.,AryanPandey,Astrophysics,Comment,6
4,"again, the movement needs to propagate through...",Lewri,Astrophysics,Comment,24
...,...,...,...,...,...
9751,[https://www.lanl.gov/projects/national-securi...,WorriedPurpose,Quantum,Comment,1
9752,"Yeah, just thought that post was particularly ...",SaltKick2,Quantum,Comment,33
9753,‘Internships’ are rarely advertised. Find prof...,youngeverest,Quantum,Comment,69
9754,I bet the quantum internet people at TUDelf an...,rrtucci,Quantum,Comment,23


In [19]:
all_comm_df = all_comm_df.rename(columns={'body':'combo_text'})
all_comm_df

,combo_text,author,Category,Post_comment,word_count
0,Your mum says your pole is more like 2 inches ...,moon-worshiper,Astrophysics,Comment,67
1,"&gt; 760 mph\n\nJust to clarify, thats the spe...",Lewri,Astrophysics,Comment,19
2,Threw this git repo together [https://github.c...,physmathastro,Astrophysics,Comment,40
3,oh okay thanks for clearing that.,AryanPandey,Astrophysics,Comment,6
4,"again, the movement needs to propagate through...",Lewri,Astrophysics,Comment,24
...,...,...,...,...,...
9751,[https://www.lanl.gov/projects/national-securi...,WorriedPurpose,Quantum,Comment,1
9752,"Yeah, just thought that post was particularly ...",SaltKick2,Quantum,Comment,33
9753,‘Internships’ are rarely advertised. Find prof...,youngeverest,Quantum,Comment,69
9754,I bet the quantum internet people at TUDelf an...,rrtucci,Quantum,Comment,23


In [13]:
all_comm_df = all_comm_df.reindex(columns=['combo_text','Category', 'Post_comment', 'word_count','author'])
all_comm_df

,combo_text,Category,Post_comment,word_count,author
0,Your mum says your pole is more like 2 inches ...,Astrophysics,Comment,67,moon-worshiper
1,"&gt; 760 mph\n\nJust to clarify, thats the spe...",Astrophysics,Comment,19,Lewri
2,Threw this git repo together [https://github.c...,Astrophysics,Comment,40,physmathastro
3,oh okay thanks for clearing that.,Astrophysics,Comment,6,AryanPandey
4,"again, the movement needs to propagate through...",Astrophysics,Comment,24,Lewri
...,...,...,...,...,...
9751,[https://www.lanl.gov/projects/national-securi...,Quantum,Comment,1,WorriedPurpose
9752,"Yeah, just thought that post was particularly ...",Quantum,Comment,33,SaltKick2
9753,‘Internships’ are rarely advertised. Find prof...,Quantum,Comment,69,youngeverest
9754,I bet the quantum internet people at TUDelf an...,Quantum,Comment,23,rrtucci


In [21]:
all_comm_df['combo_text'] = all_comm_df['combo_text'].replace('[^\w ]','',regex=True).astype(str)
all_comm_df

,combo_text,author,Category,Post_comment,word_count
0,Your mum says your pole is more like 2 inches ...,moon-worshiper,Astrophysics,Comment,67
1,gt 760 mphJust to clarify thats the speed of s...,Lewri,Astrophysics,Comment,19
2,Threw this git repo together httpsgithubcomSha...,physmathastro,Astrophysics,Comment,40
3,oh okay thanks for clearing that,AryanPandey,Astrophysics,Comment,6
4,again the movement needs to propagate through ...,Lewri,Astrophysics,Comment,24
...,...,...,...,...,...
9751,httpswwwlanlgovprojectsnationalsecurityeducati...,WorriedPurpose,Quantum,Comment,1
9752,Yeah just thought that post was particularly f...,SaltKick2,Quantum,Comment,33
9753,Internships are rarely advertised Find profess...,youngeverest,Quantum,Comment,69
9754,I bet the quantum internet people at TUDelf an...,rrtucci,Quantum,Comment,23


In [22]:
all_comm_df['Category'].replace({'Astrophysics':1,'Quantum':0},inplace=True)
all_comm_df

,combo_text,author,Category,Post_comment,word_count
0,Your mum says your pole is more like 2 inches ...,moon-worshiper,1,Comment,67
1,gt 760 mphJust to clarify thats the speed of s...,Lewri,1,Comment,19
2,Threw this git repo together httpsgithubcomSha...,physmathastro,1,Comment,40
3,oh okay thanks for clearing that,AryanPandey,1,Comment,6
4,again the movement needs to propagate through ...,Lewri,1,Comment,24
...,...,...,...,...,...
9751,httpswwwlanlgovprojectsnationalsecurityeducati...,WorriedPurpose,0,Comment,1
9752,Yeah just thought that post was particularly f...,SaltKick2,0,Comment,33
9753,Internships are rarely advertised Find profess...,youngeverest,0,Comment,69
9754,I bet the quantum internet people at TUDelf an...,rrtucci,0,Comment,23


#### Set up everything

In [24]:
# X and y
X = all_comm_df['combo_text']
y = all_comm_df['Category']

X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y)

In [25]:
# Instantiate a CountVectorizer with the default hyperparameters.
cvec = CountVectorizer(min_df=2)

# Fit the vectorizer on our corpus.
cvec.fit(X_train)

# Transform the corpus.
X_train_cvec = cvec.transform(X_train)

#transform the test data
X_test_cvec = cvec.transform(X_test)

In [26]:
len(cvec.get_feature_names())

12904

In [32]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=20)
dtc.fit(X_train_cvec,y_train)

dtc.score(X_train_cvec,y_train), dtc.score(X_test_cvec,y_test)

(0.7285644971234491, 0.7195426195426196)

In [29]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=200)
abc.fit(X_train_cvec,y_train)
abc.score(X_train_cvec,y_train), abc.score(X_test_cvec,y_test)

(0.8104942122409371, 0.7941787941787942)